<a href="https://colab.research.google.com/github/mohitsaini65/Fraud-News-Detection-System/blob/main/Fraud_News_Detection_System_BiLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount Google Drive (if not already mounted)
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import zipfile
import os

zip_path = "/content/drive/MyDrive/Colab Notebooks/Fraud-News-Detection-System/archive (1).zip"
extract_path = "/content/drive/MyDrive/Colab Notebooks/Fraud-News-Detection-System/data"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Unzipped successfully")


Unzipped successfully


In [ ]:
os.listdir("/content/drive/MyDrive/Colab Notebooks/Fraud-News-Detection-System/data")

['News _dataset']

In [ ]:
os.listdir("/content/drive/MyDrive/Colab Notebooks/Fraud-News-Detection-System/data/News _dataset")


['Fake.csv', 'True.csv']

In [ ]:
os.listdir("/content/drive/MyDrive/Colab Notebooks/Fraud-News-Detection-System/data/News _dataset")

['Fake.csv', 'True.csv']

In [ ]:
import pandas as pd

path = "/content/drive/MyDrive/Colab Notebooks/Fraud-News-Detection-System/data/News _dataset/"

fake = pd.read_csv(path + "Fake.csv")
true = pd.read_csv(path + "True.csv")

print(fake.shape, true.shape)


(23481, 4) (21417, 4)


In [ ]:
fake['label'] = 0   # Fake news
true['label'] = 1   # Real news

In [ ]:
import pandas as pd

df = pd.concat([fake, true], axis=0)
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

df.head()

,title,text,subject,date,label
0,Ben Stein Calls Out 9th Circuit Court: Committ...,"21st Century Wire says Ben Stein, reputable pr...",US_News,"February 13, 2017",0
1,Trump drops Steve Bannon from National Securit...,WASHINGTON (Reuters) - U.S. President Donald T...,politicsNews,"April 5, 2017",1
2,Puerto Rico expects U.S. to lift Jones Act shi...,(Reuters) - Puerto Rico Governor Ricardo Rosse...,politicsNews,"September 27, 2017",1
3,OOPS: Trump Just Accidentally Confirmed He Le...,"On Monday, Donald Trump once again embarrassed...",News,"May 22, 2017",0
4,Donald Trump heads for Scotland to reopen a go...,"GLASGOW, Scotland (Reuters) - Most U.S. presid...",politicsNews,"June 24, 2016",1


In [ ]:
df = df[['text', 'label']]
df.dropna(inplace=True)

df.head(), df.shape

(                                                text  label
 0  21st Century Wire says Ben Stein, reputable pr...      0
 1  WASHINGTON (Reuters) - U.S. President Donald T...      1
 2  (Reuters) - Puerto Rico Governor Ricardo Rosse...      1
 3  On Monday, Donald Trump once again embarrassed...      0
 4  GLASGOW, Scotland (Reuters) - Most U.S. presid...      1,
 (44898, 2))

In [ ]:
X = df['text']
y = df['label']

print(X.shape, y.shape)


(44898,) (44898,)


In [ ]:
import re
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

corpus = []

for text in X:
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = text.lower()
    words = text.split()
    words = [w for w in words if w not in stop_words]
    corpus.append(' '.join(words))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from tensorflow.keras.preprocessing.text import one_hot

voc_size = 5000
onehot_repr = [one_hot(words, voc_size) for words in corpus]

onehot_repr[:2]


[[4158,
  3238,
  4206,
  1019,
  3576,
  3923,
  2172,
  4401,
  2565,
  3808,
  4002,
  676,
  1631,
  2081,
  3420,
  3551,
  3387,
  4904,
  2234,
  4345,
  2927,
  2972,
  4886,
  3586,
  2510,
  3716,
  4051,
  4199,
  3693,
  4635,
  4276,
  3630,
  3204,
  237,
  15,
  2460,
  3923,
  3150,
  3857,
  4229,
  2697,
  1108,
  3325,
  168,
  3083,
  4837,
  237,
  2320,
  1492,
  3923,
  2472,
  57,
  1461,
  2492,
  1851,
  417,
  4686,
  1851,
  4002,
  1799,
  488,
  948,
  4886,
  2007,
  46,
  786,
  88,
  3770,
  1308,
  637,
  3669,
  2007,
  1108,
  3374,
  2115,
  3750,
  4413,
  4169,
  654,
  711,
  1375,
  2065,
  4635,
  3434,
  4587,
  237,
  15,
  556,
  3204,
  3619,
  4158,
  3238,
  4206,
  3204,
  1614,
  3495,
  4353,
  4986,
  3475,
  4206,
  3420],
 [3325,
  4819,
  4734,
  3630,
  665,
  3204,
  2847,
  4540,
  1993,
  3835,
  618,
  1375,
  1928,
  885,
  4707,
  2884,
  3530,
  3905,
  4572,
  1512,
  3014,
  1851,
  3339,
  2716,
  2159,
  1928,
  4172,
 

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

sent_length = 20
embedded_docs = pad_sequences(
    onehot_repr,
    padding='pre',
    maxlen=sent_length
)

embedded_docs.shape


(44898, 20)

In [ ]:
import numpy as np

X_final = np.array(embedded_docs)
y_final = np.array(y)

X_final.shape, y_final.shape


((44898, 20), (44898,))

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_final,
    y_final,
    test_size=0.33,
    random_state=42
)

X_train.shape, X_test.shape


((30081, 20), (14817, 20))

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense, Dropout

embedding_vector_features = 40

model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=10,
    batch_size=64
)


Epoch 1/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 36s 67ms/step - accuracy: 0.8640 - loss: 0.2860 - val_accuracy: 0.9453 - val_loss: 0.1552
Epoch 2/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 32s 67ms/step - accuracy: 0.9680 - loss: 0.0943 - val_accuracy: 0.9543 - val_loss: 0.1291
Epoch 3/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 40s 65ms/step - accuracy: 0.9817 - loss: 0.0569 - val_accuracy: 0.9494 - val_loss: 0.1468
Epoch 4/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 32s 67ms/step - accuracy: 0.9716 - loss: 0.0817 - val_accuracy: 0.9523 - val_loss: 0.1666
Epoch 5/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 32s 67ms/step - accuracy: 0.9912 - loss: 0.0282 - val_accuracy: 0.9498 - val_loss: 0.1819
Epoch 6/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 31s 65ms/step - accuracy: 0.9936 - loss: 0.0218 - val_accuracy: 0.9519 - val_loss: 0.1925
Epoch 7/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 31s 66ms/step - accuracy: 0.9957 - loss: 0.0149 - val_accuracy: 0.9491 - val_loss: 0.2188
Epoch 8/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 36s 77ms/step - accuracy: 0.9965 - loss: 0.0128 - 

In [ ]:
y_pred = (model.predict(X_test) > 0.5).astype(int)

464/464 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step


In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[7440,  314],
       [ 408, 6655]])

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.96      0.95      7754
           1       0.95      0.94      0.95      7063

    accuracy                           0.95     14817
   macro avg       0.95      0.95      0.95     14817
weighted avg       0.95      0.95      0.95     14817



In [ ]:
model.build(input_shape=(None, sent_length))
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 20, 40)         │       200,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 200)            │       112,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 200)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           201 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 313,001 (1.19 MB)

 Trainable params: 313,001 (1.19 MB)

 Non-trainable params: 0 (0.00 B)